# Exam of 24th Feb 2020

## Part 1 - MCQ

1. d - All other answers are wrong
2. d - The output of each batch does not depend on other batches

## Part 2 - Exercises

### Ex 1

In [18]:
!hdfs dfs -rm -r /user/s315054/exam_sim/ex_2020_02_24/ex1

23/02/01 14:42:50 INFO fs.TrashPolicyDefault: Moved: 'hdfs://BigDataHA/user/s315054/exam_sim/ex_2020_02_24/ex1' to trash at: hdfs://BigDataHA/user/s315054/.Trash/Current/user/s315054/exam_sim/ex_2020_02_24/ex11675262570370


In [19]:
# With DFs

#import sys

out_1 = '/user/s315054/exam_sim/ex_2020_02_24/ex1'
in_a = '/data/students/bigdata_internet/exam_examples_data/exam20200221_data/Bicycles.txt'
in_b = '/data/students/bigdata_internet/exam_examples_data/exam20200221_data/Bicycles_Unavailable.txt'

unav_DF = spark.read.load(in_b, format='csv', sep=',', header=False)\
                .toDF('BID', 'from_ts', 'to_ts')
unav_DF.createOrReplaceTempView('unavail_table')

bikes_DF = spark.read.load(in_a, format='csv', sep=',', header=False)\
                .toDF('BID', 'manufacturer', 'city', 'country')
bikes_DF.createOrReplaceTempView('bikes_table')

tmp_DF = spark.sql("""
        SELECT unavail_table.BID
        FROM unavail_table
        GROUP BY unavail_table.BID
        HAVING count(*) >= 100
""")

out_1_DF = tmp_DF.join(bikes_DF.select('BID', 'manufacturer'), 'BID')

out_1_DF.write.csv(out_1, header=False)

In [23]:
unav_DF.select('BID').distinct().count()

318

In [24]:
unav_DF.select('BID').count()

998

### Ex 2.1

In [ ]:
!hdfs dfs -rm -r /user/s315054/exam_sim/ex_2020_02_24/ex2_1

In [26]:
# DFs

in_a = '/data/students/bigdata_internet/exam_examples_data/exam20200221_data/Bicycles.txt'
in_b = '/data/students/bigdata_internet/exam_examples_data/exam20200221_data/Bicycles_Unavailable.txt'

out_2_1 = '/user/s315054/exam_sim/ex_2020_02_24/ex2_1'
out_2_2 = '/user/s315054/exam_sim/ex_2020_02_24/ex2_2'

unav_DF = spark.read.load(in_b, format='csv', header=False)\
                .toDF('BID', 'from_ts', 'to_ts')
unav_DF.createOrReplaceTempView('unavail_tab')

bikes_DF = spark.read.load(in_a, format='csv', header=False)\
                .toDF('BID', 'manufacturer', 'city', 'country')

# UDF for getting the year from the timestamp:
spark.udf.register('getYear', lambda ts: int(ts.split('/')[0]))

# Keep unavailable bikes having both dates in 2018
# Count how many records for each BID in 2018 and keep the ones having at least 5
# Each BID will just appear once here
n_unavail_DF = spark.sql("""
            SELECT BID, count(*)
            FROM unavail_tab
            WHERE getYear(from_ts) == 2018 AND getYear(to_ts) == 2018
            GROUP BY BID
            HAVING count(*) >= 5
""")

italian_cities_DF = bikes_DF.filter('country == "Italy"')

with_cities_DF = n_unavail_DF.join(italian_cities_DF.select('BID', 'city'), 'BID').groupBy('city').agg({'BID':'count'})\
                                .withColumnRenamed('count(BID)', 'n_crit_bikes').filter('n_crit_bikes >= 100')

with_cities_DF.select('city').write.csv(out_2_1, header=False)

23/02/01 14:45:59 WARN analysis.SimpleFunctionRegistry: The function getyear replaced a previously registered function.


In [31]:
with_cities_DF.show()

+----+------------+
|city|n_crit_bikes|
+----+------------+
+----+------------+



### Ex 2.2

In [ ]:
!hdfs dfs -rm -r /user/s315054/exam_sim/ex_2020_02_24/ex2_2

In [29]:
# DFs from before

# Get list of all cities:
all_ita_cities_RDD = italian_cities_DF.select('city').distinct().rdd.map(lambda el: el.city)
# Each element of the RDD is a LIST (length=1)


# Work on the unavailable bikes

# Filter for march 2019
spark.udf.register('getMonth', lambda ts: int(ts.split('/')[1]))

# Isolate records form march '19
# Distinct at the end allows to only get a single element per BID - it is enough
march_19_DF = spark.sql("""
            SELECT unavail_tab.BID
            FROM unavail_tab
            WHERE getYear(from_ts) == 2019 AND getYear(to_ts) == 2019 AND getMonth(from_ts) == 3 AND getMonth(to_ts) == 3
""").distinct()

# Get Italian city associated with each 'unavailable' BID - extract the city and remove duplicates
unavail_cities_19_DF = march_19_DF.join(italian_cities_DF, 'BID')
unavail_cities_RDD = unavail_cities_19_DF.select('city').distinct().rdd.map(lambda el: el.city)

# Subtract bad cities from initial set - get cities with no unavailable bikes in march 2019
okay_cities = all_ita_cities_RDD.subtract(unavail_cities_RDD)

okay_cities.saveAsTextFile(out_2_2)


23/02/01 14:46:52 WARN analysis.SimpleFunctionRegistry: The function getmonth replaced a previously registered function.


In [33]:
okay_cities.collect()

['Bologna']

In [34]:
all_ita_cities_RDD.collect()

['Bologna', 'Venice', 'Milan', 'Rome', 'Turin']

In [37]:
italian_cities_DF.select('city').distinct().rdd.map(lambda el: el.city).collect()

['Bologna', 'Venice', 'Milan', 'Rome', 'Turin']